# Visualizing Networks

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

### Data

### Import edges and nodes into networkx 2.X

<div class=warn>Networkx 1.X files cannot be read by networkx 2.X</div>

```python
file = './major_us_cities.dms'
G = nx.read_gpickle(file)
print(nx.info(G))

list(G.edges(data=True))[:10]
list(G.nodes(data=True))[:10]

edges = pd.DataFrame(list(G.edges(data=True)))
nodes = pd.DataFrame(list(G.nodes(data=True)))
nodes.sample()
edges.sample()

# Use a lambda to pull out the attributes from the attributes dictionary in column 1
nodes['location'] = nodes.loc[:, 1].map(lambda x: x['location'])
nodes['population'] = nodes.loc[:, 1].map(lambda x: x['population'])
del nodes[1]
nodes.sample()

# Use a lambda to pull out the attributes from the attributes dictionary in column 1
edges['weight'] = edges.loc[:, 2].map(lambda x: x['weight'])
del edges[2]
edges.sample()

edges.to_csv('major_us_cities_edges.csv')
nodes.to_csv('major_us_cities_email_nodes.csv')
```

In [ ]:
!find .. | grep -i major_us_cities

In [ ]:
nodes = pd.read_csv('../_data/major_us_cities_nodes.csv', index_col=0, 
                    names=['node', 'location', 'population'])
edges = pd.read_csv('../_data/major_us_cities_edges.csv', index_col=0,
                   names=['n1', 'n2', 'weight'])
nodes.head()
nodes.info()
edges.head()
edges.info()

#### Remove header

In [ ]:
nodes = nodes.iloc[1:, :]
edges = edges.iloc[1:, :]

#### Convert weight, location and population to numeric

In [ ]:
edges['weight'] = [float(x) for x in edges.weight]

In [ ]:
nodes['population'] = [int(x) for x in nodes.population]

In [ ]:
import re
nodes['location'] = [(float(re.sub('\(', '', x.split(',')[0])), float(re.sub('\)', '', x.split(',')[1]))) for x in nodes.location]

#### Create graph from edges and then add node attributes

In [ ]:
G = nx.from_pandas_dataframe(edges, 'n1', 'n2', edge_attr='weight')
print(nx.info(G))

In [ ]:
list(G.edges(data=True))[:5]
list(G.nodes(data=True))[:5]

# remove header
try:
    G.remove_node('0')
    G.remove_node('1')
    G.remove_edge('0', '1')
except:
    None

In [ ]:
_ = [G.add_node(nodes.loc[n, 'node'], 
                location=nodes.loc[n, 'location'], 
                population=nodes.loc[n, 'population']) for n in nodes.index]

In [ ]:
list(G.nodes(data=True))[:10]

In [ ]:
# draw the graph using the default spring layout
plt.figure(figsize=(10,9))
nx.draw_networkx(G);

In [ ]:
# See what layouts are available in networkX
[x for x in nx.__dir__() if x.endswith('_layout')]

In [ ]:
# Draw the graph using the random layout
plt.figure(figsize=(10,9))
pos = nx.random_layout(G)
nx.draw_networkx(G, pos);

In [ ]:
# Draw the graph using the circular layout
plt.figure(figsize=(10,9))
pos = nx.circular_layout(G)
nx.draw_networkx(G, pos);

#### Use geo position

In [ ]:
pos = [loc for loc in nx.get_node_attributes(G, 'location').values()]
pos[:5]

In [ ]:
# Draw the graph using custom node positions
plt.figure(figsize=(10,7))

pos = nx.get_node_attributes(G, 'location')
nx.draw_networkx(G, pos)
plt.axis('off');

In [ ]:
# Draw the graph adding alpha, removing labels, and softening edge color
plt.figure(figsize=(10,7))

nx.draw_networkx(G, 
                 pos, 
                 alpha=0.7, 
                 with_labels=False, 
                 edge_color='.4')

plt.axis('off')
plt.tight_layout();

In [ ]:
# Draw graph with varying node color, node size, and edge width
plt.figure(figsize=(10,7))

node_color = [G.degree(v) for v in G]
node_size = [0.0005*nx.get_node_attributes(G, 'population')[v] for v in G]
edge_width = [0.0015*G[u][v]['weight'] for u,v in G.edges()]

nx.draw_networkx(G, pos, node_size=node_size, 
                 node_color=node_color, alpha=0.7, with_labels=False, 
                 width=edge_width, edge_color='.4', cmap=plt.cm.Blues)

plt.axis('off')
plt.tight_layout();

In [ ]:
# Draw specific edges and add labels to specific nodes
plt.figure(figsize=(10,7))

node_color = [G.degree(v) for v in G]
node_size = [0.0005*nx.get_node_attributes(G, 'population')[v] for v in G]
edge_width = [0.0015*G[u][v]['weight'] for u,v in G.edges()]

nx.draw_networkx(G, pos, node_size=node_size, 
                 node_color=node_color, alpha=0.7, with_labels=False, 
                 width=edge_width, edge_color='.4', cmap=plt.cm.Blues)


greater_than_770 = [x for x in G.edges(data=True) if x[2]['weight']>770]
nx.draw_networkx_edges(G, pos, edgelist=greater_than_770, edge_color='r', alpha=0.4, width=6)

nx.draw_networkx_labels(G, pos, labels={'Los Angeles, CA': 'LA', 'New York, NY': 'NYC'}, font_size=18, font_color='w')

plt.axis('off')
plt.tight_layout();

#### Use cartopy to project the points on the map:

In [ ]:
import cartopy.crs as ccrs

In [ ]:
# Map projection
crs = ccrs.PlateCarree()  #Mercator() #Geodetic() #Mollweide() #PlateCarree()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 20), subplot_kw=dict(projection=crs))
ax.coastlines()

# Extent of continental US. (l,r,b,t)
ax.set_extent([-125, -65, 20, 50])

node_color = [G.degree(v) for v in G]
node_size = [0.0005*nx.get_node_attributes(G, 'population')[v] for v in G]
edge_width = [0.0015*G[u][v]['weight'] for u,v in G.edges()]

nx.draw_networkx(G, pos, node_size=node_size, 
                 node_color=node_color, alpha=0.7, with_labels=False, 
                 width=edge_width, edge_color='.4', cmap=plt.cm.Blues)


greater_than_770 = [x for x in G.edges(data=True) if x[2]['weight']>770]
nx.draw_networkx_edges(G, pos, edgelist=greater_than_770, edge_color='r', alpha=0.4, width=5)

nx.draw_networkx_labels(G, pos, labels={'Los Angeles, CA': 'LA', 'New York, NY': 'NYC'}, font_size=18, font_color='w')
ax.axis('off')
plt.tight_layout();